In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
##Analyze the ventillator pressure dataset
import pandas as pd
import numpy as np
##Create a NN model
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping


trainDataPath = "/kaggle/input/ventilator-pressure-prediction/train.csv"
df_data = pd.read_csv(trainDataPath, header=0)

## Create new integer timesteps for the data.
##New timeStep performs worse, trying normalization
#df_data["newTimeStep"] = df_data.groupby("breath_id").cumcount()

df_data.head()
#df_data.describe()

2021-10-06 13:58:54.056443: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-06 13:58:54.056585: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [2]:
"""
## Change column order after new timeStep is added
df_data.columns
df_data = df_data[['id', 'breath_id', 'time_step', 'R', 'C', 'newTimeStep', 'u_in', 'u_out', 'pressure']]
df_data.head()
"""
## Add prevPressure
df_data["prevPressure"] = df_data["pressure"].shift(periods=1, fill_value=0)
index_0 = df_data.loc[df_data["time_step"] == 0].index
df_data.loc[index_0]["prevPressure"] = 0
df_data.iloc[index_0,-1] = 0
df_data.loc[75:85,:]
df_data.columns
df_data = df_data[['id', 'breath_id', 'R', 'C', 'time_step', 'u_in', 'u_out','prevPressure','pressure']]
df_data.head()

,id,breath_id,R,C,time_step,u_in,u_out,prevPressure,pressure
0,1,1,20,50,0.000000,0.083334,0,0.000000,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.837492,5.907794
2,3,1,20,50,0.067514,22.509278,0,5.907794,7.876254
3,4,1,20,50,0.101542,22.808822,0,7.876254,11.742872
4,5,1,20,50,0.135756,25.355850,0,11.742872,12.234987


In [3]:
## Train a FNN for baseline
from sklearn.model_selection import train_test_split

## Remove random_state if we want variable train and dev sizes
df_train, df_dev = train_test_split(df_data, train_size=0.8, random_state=9)
print("data shape= %s, train size= %s, dev size = %s\n" %(df_data.shape, df_train.shape, df_dev.shape))

df_data_x = df_data.iloc[:,2:8]

##Get features we will use for training
df_train_x = df_train.iloc[:,2:8] ##[:,2:7] for original data
df_train_y = df_train.iloc[:,-1]

print("train_x= %s, train_y= %s" %(df_train_x.shape, df_train_y.shape))
print("train= \n%s" %(df_train_x.head() ) )
df_train_y.head()

##Get the features for dev set
df_dev_x = df_dev.iloc[:,2:8]
df_dev_y = df_dev.iloc[:,-1]
df_dev_x.head()

##Test set
testPath = "/kaggle/input/ventilator-pressure-prediction/test.csv"
df_test = pd.read_csv(testPath, header=0)
df_test_x = df_test.iloc[:,2:8]
df_test_x.head()

data shape= (6036000, 9), train size= (4828800, 9), dev size = (1207200, 9)

train_x= (4828800, 6), train_y= (4828800,)
train= 
          R   C  time_step      u_in  u_out  prevPressure
2392615  20  50   1.871268  4.330520      1      7.454441
3623600  50  10   0.000000  3.734260      0      0.000000
5062600  50  50   1.279855  0.000000      1      6.610815
1485964  20  50   1.518733  0.647814      1      6.118700
5452762  50  10   1.330573  0.000000      1      6.470211


,R,C,time_step,u_in,u_out
0,5,20,0.000000,0.000000,0
1,5,20,0.031904,7.515046,0
2,5,20,0.063827,14.651675,0
3,5,20,0.095751,21.230610,0
4,5,20,0.127644,26.320956,0


In [18]:
##Normalize training data
df_data.iloc[:,2:7].max()
data_min = df_data.iloc[:,2:7].min(axis=0)
data_max = df_data.iloc[:,2:7].max(axis=0)

df_data_norm = (df_data_x - data_min) / (data_max-data_min)

df_train_x_norm = (df_train_x - data_min ) / (data_max-data_min)
df_dev_x_norm = (df_dev_x - data_min) / (data_max-data_min)
df_dev_x_norm.head(10)

df_test_x_norm = (df_test_x - data_min) / (data_max-data_min)
df_test_x_norm.head()
#train_min

,R,C,time_step,u_in,u_out
0,0.0,0.25,0.000000,0.000000,0.0
1,0.0,0.25,0.010862,0.075150,0.0
2,0.0,0.25,0.021730,0.146517,0.0
3,0.0,0.25,0.032599,0.212306,0.0
4,0.0,0.25,0.043457,0.263210,0.0


In [4]:
##Create a NN model
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping

##Clear models
tf.keras.backend.clear_session()

model = Sequential()
model.add(tf.keras.Input(shape=(6,)))
model.add(Dense(32,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 12,769
Trainable params: 12,769
Non-trainable params: 0
_________________________________________________________________


2021-10-06 14:00:20.840941: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-06 14:00:20.844663: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-06 14:00:20.844705: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-06 14:00:20.844733: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (316b47e8018c): /proc/driver/nvidia/version does not exist
2021-10-06 14:00:20.845132: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [5]:
## Earlystopping
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True,\
                                             verbose=1)

##Fit model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])

##Train the model
history = model.fit(x=df_train_x, y=df_train_y, batch_size=1024, epochs=40, validation_data=(df_dev_x, df_dev_y),\
                   callbacks=[earlystop])

2021-10-06 14:00:34.442536: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-06 14:00:34.458407: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Epoch 1/40
4716/4716 [==============================] - 24s 5ms/step - loss: 7.2902 - mean_absolute_error: 1.1899 - val_loss: 3.3111 - val_mean_absolute_error: 0.8655
Epoch 2/40
4716/4716 [==============================] - 22s 5ms/step - loss: 3.2439 - mean_absolute_error: 0.8566 - val_loss: 3.0417 - val_mean_absolute_error: 0.8006
Epoch 3/40
4716/4716 [==============================] - 22s 5ms/step - loss: 3.0595 - mean_absolute_error: 0.8151 - val_loss: 2.9785 - val_mean_absolute_error: 0.8043
Epoch 4/40
4716/4716 [==============================] - 23s 5ms/step - loss: 2.9614 - mean_absolute_error: 0.7971 - val_loss: 2.8340 - val_mean_absolute_error: 0.7586
Epoch 5/40
4716/4716 [==============================] - 22s 5ms/step - loss: 2.8744 - mean_absolute_error: 0.7833 - val_loss: 2.6595 - val_mean_absolute_error: 0.7462
Epoch 6/40
4716/4716 [==============================] - 22s 5ms/step - loss: 2.6716 - mean_absolute_error: 0.7559 - val_loss: 2.3977 - val_mean_absolute_error: 0.709

In [6]:
## Save the model before prediction
model.save("./ventilator_fnn_prevPressure.h5")
##Evaluate model on dev set
score = model.evaluate(df_dev_x, df_dev_y, batch_size=10240)
score

118/118 [==============================] - 1s 6ms/step - loss: 1.6888 - mean_absolute_error: 0.5942


[1.68878173828125, 0.5942315459251404]

In [77]:
##Get the predictions
#df_test["newTimeStep"] = df_test.groupby("breath_id").cumcount()
"""
df_test.columns
df_test = df_test[['id', 'breath_id', 'time_step', 'R', 'C', 'u_in', 'u_out','newTimeStep']]
df_test.head()
df_test_x = df_test.iloc[:,3:8]
df_test_x.head()
"""
##Load model
#model= load_model("/kaggle/input/lung1/kaggle/working/ventilator_fnn_1.h5")
predict = model.predict(df_test_x, batch_size=10240, verbose=1)

393/393 [==============================] - 1s 2ms/step


In [118]:
##Prediction for prevPressure
pressPredict = []
## Adding prediction column before we iterate so that we can access it in each row
print(df_test_x.columns)
print("%s" %(df_test_x.reindex(columns=['R', 'C', 'time_step', 'u_in', 'u_out', 'prevPressure', 'prediction'], fill_value=0).head()) )
print(type(df_test_x))
print(df_data_x.loc[0])
print("predict_train= %s" %(model.predict(np.array([df_data_x.loc[0]]).reshape(1,6))))
df_test_x = df_test_x.reindex(columns=['R', 'C', 'time_step', 'u_in', 'u_out', 'prevPressure', 'prediction'], fill_value=0)
for i, row in enumerate(df_test_x.loc[:240,:].itertuples(index=False)):
    if(row.time_step == 0):
        print(row[0:6])
        print(model.predict(np.array(row[:6]).reshape(1,6)))
        predict = model.predict(np.array(row[:6]).reshape(1,6))
        print(dir(row))
        #row["prediction"] = predict
        #setattr(row, "prediction", predict)
       

Index(['R', 'C', 'time_step', 'u_in', 'u_out', 'prevPressure', 'prediction'], dtype='object')
   R   C  time_step       u_in  u_out  prevPressure  prediction
0  5  20   0.000000   0.000000      0             0           0
1  5  20   0.031904   7.515046      0             0           0
2  5  20   0.063827  14.651675      0             0           0
3  5  20   0.095751  21.230610      0             0           0
4  5  20   0.127644  26.320956      0             0           0
<class 'pandas.core.frame.DataFrame'>
R               20.000000
C               50.000000
time_step        0.000000
u_in             0.083334
u_out            0.000000
prevPressure     0.000000
Name: 0, dtype: float64
predict_train= [[6.1120405]]
(5, 20, 0.0, 0.0, 0, 0)
[[6.510953]]
['C', 'R', '__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter_

AttributeError: 'int' object has no attribute '_replace'

In [32]:
import time
##Get predictions for prevPressure models
trial = df_data.iloc[:160,:-2]

#trial = df_test
print("trialShape= %s" %(trial.shape,))
trial["newTimeStep"] = trial.groupby("breath_id").cumcount()
trial = trial.reindex(columns=['id','breath_id','newTimeStep', 'R', 'C', 'time_step', 'u_in', 'u_out','prevPressure', 'prediction'],\
                      fill_value=0)

for predIdx in range(85):
    startLoopTime = time.time()
    print("preIdx= %s" %(predIdx))
    
    index_to_predict = trial.loc[trial["newTimeStep"] == predIdx].index
    #print(len(index_to_predict))
    
    if(len(index_to_predict) is not 0):
        trial["prevPressure"] = trial["prediction"].shift(periods=1, fill_value=0)
        predictionChunk = trial.iloc[index_to_predict,3:9]
        #print(predictionChunk)
        predictedChunk = model.predict(predictionChunk, batch_size=2)
        #print(predictedChunk)
        trial.loc[index_to_predict, "prediction"] = predictedChunk
        #trial["prediction"] = trial.apply(lambda x: model.predict(np.array(x[3:9]).reshape(1,6))[0][0] if x["newTimeStep"]==0 else 0, axis=1)   
    endLoopTime = time.time()
    print("Time taken for predIdx= %s, is %s sec" %(predIdx, endLoopTime-startLoopTime))
print(trial.head())
#print(trial.tail())
#print(trial.iloc[75:85,:])

"""
##Compare with actual data
trial["actualPressure"] = df_data.iloc[:81]["pressure"]
trial.tail(80)
"""

trialShape= (160, 7)
preIdx= 0
Time taken for predIdx= 0, is 0.04369854927062988 sec
preIdx= 1
Time taken for predIdx= 1, is 0.04679250717163086 sec
preIdx= 2
Time taken for predIdx= 2, is 0.049541473388671875 sec
preIdx= 3
Time taken for predIdx= 3, is 0.045082807540893555 sec
preIdx= 4
Time taken for predIdx= 4, is 0.05742073059082031 sec
preIdx= 5
Time taken for predIdx= 5, is 0.046187400817871094 sec
preIdx= 6
Time taken for predIdx= 6, is 0.04604363441467285 sec
preIdx= 7
Time taken for predIdx= 7, is 0.046207427978515625 sec
preIdx= 8
Time taken for predIdx= 8, is 0.04417920112609863 sec
preIdx= 9
Time taken for predIdx= 9, is 0.045859575271606445 sec
preIdx= 10
Time taken for predIdx= 10, is 0.0434722900390625 sec
preIdx= 11
Time taken for predIdx= 11, is 0.045380353927612305 sec
preIdx= 12
Time taken for predIdx= 12, is 0.044010162353515625 sec
preIdx= 13
Time taken for predIdx= 13, is 0.044315338134765625 sec
preIdx= 14
Time taken for predIdx= 14, is 0.07037544250488281 sec
pr

'\n##Compare with actual data\ntrial["actualPressure"] = df_data.iloc[:81]["pressure"]\ntrial.tail(80)\n'

In [231]:
trial.head()
submission_csv = pd.DataFrame(columns=['id', 'pressure'])
submission_csv["id"] = trial["id"]
submission_csv["pressure"] = trial["prediction"]
submission_csv.to_csv("./ventilator_fnn_prevPressure.csv", index=False)
print("Done saving file")

Done saving file


TypeError: 'tuple' object is not callable

In [233]:
df_test.shape

(4024000, 7)

In [78]:
type(predict)
predict[-100:-90]
df_test.tail()

,id,breath_id,R,C,time_step,u_in,u_out
4023995,4023996,125748,20,10,2.530117,4.971245,1
4023996,4023997,125748,20,10,2.563853,4.975709,1
4023997,4023998,125748,20,10,2.597475,4.979468,1
4023998,4023999,125748,20,10,2.631134,4.982648,1
4023999,4024000,125748,20,10,2.665301,4.985373,1


In [80]:
df_test_predict=pd.DataFrame()
df_test_predict=pd.concat([df_test, pd.DataFrame(data=predict, columns=['pressure'])], axis=1)
print("%s" %(df_test_predict.tail() ))

##Submission csv
submission_csv = pd.DataFrame(columns=['id', 'pressure'])
submission_csv.head()
submission_csv = pd.concat([df_test_predict.iloc[:,0], df_test_predict.iloc[:,-1]], axis=1)
submission_csv.head()
#submission_csv.shape

submission_csv.to_csv("./ventilator_fnn_norm.csv", index=False)
print("Done saving file")

              id  breath_id   R   C  time_step      u_in  u_out  pressure
4023995  4023996     125748  20  10   2.530117  4.971245      1  6.342644
4023996  4023997     125748  20  10   2.563853  4.975709      1  6.321092
4023997  4023998     125748  20  10   2.597475  4.979468      1  6.299598
4023998  4023999     125748  20  10   2.631134  4.982648      1  6.278068
4023999  4024000     125748  20  10   2.665301  4.985373      1  6.256202
Done saving file


In [160]:
print("train shape= %s, data shape= %s" %(df_train.shape, df_data.shape))
##Load model for analysis

data_predict = model.predict(df_data_x, batch_size=10240, verbose=1)
df_data_predict = pd.concat([df_data,pd.DataFrame(data=data_predict, columns=['pressure_predict'])], axis=1)
df_data_predict.head()

train shape= (4828800, 9), data shape= (6036000, 9)
590/590 [==============================] - 4s 6ms/step


,id,breath_id,R,C,time_step,u_in,u_out,prevPressure,pressure,pressure_predict
0,1,1,20,50,0.000000,0.083334,0,0.000000,5.837492,6.112042
1,2,1,20,50,0.033652,18.383041,0,5.837492,5.907794,7.019338
2,3,1,20,50,0.067514,22.509278,0,5.907794,7.876254,8.061171
3,4,1,20,50,0.101542,22.808822,0,7.876254,11.742872,10.893771
4,5,1,20,50,0.135756,25.355850,0,11.742872,12.234987,13.404716


In [159]:
df_data_predict["absDiff"] = np.abs(df_data_predict["pressure"] - df_data_predict["pressure_predict"])
df_data_predict["percentageError"] = (df_data_predict["absDiff"]/df_data_predict["pressure"])*100
df_data_predict.head()

#df_data_predict.to_csv("./ventilator_fnn_norm_fullPredictData.csv", index=False, compression="gzip")
#print("Done saving data to csv")

,id,breath_id,R,C,time_step,u_in,u_out,prevPressure,pressure,pressure_predict,absDiff,percentageError
0,1,1,20,50,0.000000,0.083334,0,0.000000,5.837492,6.112042,0.274551,4.703231
1,2,1,20,50,0.033652,18.383041,0,5.837492,5.907794,7.019338,1.111544,18.814871
2,3,1,20,50,0.067514,22.509278,0,5.907794,7.876254,8.061171,0.184917,2.347774
3,4,1,20,50,0.101542,22.808822,0,7.876254,11.742872,10.893771,0.849101,7.230776
4,5,1,20,50,0.135756,25.355850,0,11.742872,12.234987,13.404716,1.169729,9.560522


In [9]:
df_data_predict.loc[df_data_predict["percentageError"] > 50]

,id,breath_id,R,C,time_step,u_in,u_out,prevPressure,pressure,pressure_predict,absDiff,percentageError
673,674,10,50,50,1.048312,0.000000,1,19.827619,8.157463,12.933686,4.776224,58.550361
721,722,12,50,20,0.031919,27.490064,0,6.681117,6.118700,9.792514,3.673814,60.042385
724,725,12,50,20,0.127761,0.000000,0,14.695562,9.704110,23.692743,13.988634,144.151643
725,726,12,50,20,0.159678,16.920362,0,9.704110,7.454441,12.954246,5.499805,73.778899
728,729,12,50,20,0.255294,6.199195,0,12.797404,8.719880,13.745790,5.025910,57.637376
...,...,...,...,...,...,...,...,...,...,...,...,...
6035448,6035449,125738,50,50,0.254258,15.514145,0,11.321059,8.227765,12.374789,4.147025,50.402810
6035451,6035452,125738,50,50,0.349404,0.000000,0,13.219217,9.985318,19.569221,9.583903,95.979947
6035614,6035615,125740,50,50,0.481218,0.000000,0,11.883476,9.352599,16.421400,7.068801,75.581142
6035615,6035616,125740,50,50,0.515292,0.000000,0,9.352599,7.876254,11.981740,4.105486,52.124857


In [27]:
#A=bsolute Error for each prediction
#df_data_predict["absError"] = (df_data_predict["pressure"] - df_data_predict["pressure_predict"]).abs()
#df_data_predict.head()
##Create new column with integer time-stamps
index=0
"""
for groupId, group in df_data.iloc[:240,:].groupby("breath_id")["id"]:
    #print("groupId= %s\n, group=\n%s" %(groupId, group))
    
        
    #print("type=%s\n,%s\n" %(type(group), dir(group)))
    print("%s\n" %(group))
"""
## Create new integer timesteps for the data.
df_data["newTimeStep"] = df_data.iloc[:240,:].groupby("breath_id").cumcount()
df_data.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,newTimeStep
0,1,1,20,50,0.000000,0.083334,0,5.837492,0.0
1,2,1,20,50,0.033652,18.383041,0,5.907794,1.0
2,3,1,20,50,0.067514,22.509278,0,7.876254,2.0
3,4,1,20,50,0.101542,22.808822,0,11.742872,3.0
4,5,1,20,50,0.135756,25.355850,0,12.234987,4.0


In [100]:
##Load best model
best_model = load_model("../input/lung1/kaggle/working/ventilator_fnn_1.h5")
##This was without norm
best_model.evaluate(df_train_x, df_train_y, batch_size=10240)
best_predict = best_model.predict(df_data_x, batch_size=10240)
best_data_predict = pd.concat([df_data,pd.DataFrame(data=best_predict, columns=['pressure_predict'])], axis=1)

best_data_predict["absDiff"] = np.abs(best_data_predict["pressure"] - best_data_predict["pressure_predict"])
best_data_predict["percentageError"] = (best_data_predict["absDiff"]/best_data_predict["pressure"])*100
best_data_predict.loc[best_data_predict["percentageError"] > 50]

#best_data_predict.to_csv("./ventilator_fnn_best_fullPredictData.csv", index=False, compression="gzip")
#print("Done saving data to csv")

472/472 [==============================] - 1s 3ms/step - loss: 19.1871 - mean_absolute_error: 2.3563


,id,breath_id,R,C,time_step,u_in,u_out,pressure,pressure_predict,absDiff,percentageError
111,112,2,20,20,1.054327,0.000000,1,9.844714,18.104275,8.259561,83.898433
193,194,3,50,20,1.054855,0.000000,1,23.623935,11.225804,12.398130,52.481225
429,430,6,5,50,1.000251,0.000000,1,9.844714,20.022572,10.177858,103.383984
430,431,6,5,50,1.035428,0.000000,1,9.704110,19.447685,9.743576,100.406692
431,432,6,5,50,1.069673,0.000000,1,8.016858,16.979942,8.963084,111.802952
...,...,...,...,...,...,...,...,...,...,...,...
6035792,6035793,125743,20,10,1.078994,0.000000,1,7.454441,12.064485,4.610044,61.842914
6035794,6035795,125743,20,10,1.146158,0.000000,1,5.907794,8.889663,2.981869,50.473476
6035873,6035874,125745,50,50,1.047880,0.000000,1,8.157463,12.862613,4.705150,57.679091
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729,5.724897,1.926167,50.705570


In [ ]:
()